#environment preparation
(~about 20 minutes)

In [ ]:
#@title clone git
!git clone https://github.com/Nidchapan/Closed-Domain-Chatbot.git
!gdown 1Litzuc0kKu52w-yupXqV3zEySc3WA9bU

%cd /content/Closed-Domain-Chatbot/
!mv intent_classification_weight.hdf5 weight

In [ ]:
#@title install library
!pip install openai
!pip install pythainlp[full]
!pip install pyngrok
!pip install line-bot-sdk

!pip install --no-deps thai2transformers==0.1.2
!pip install translate
!pip install tensorflow_text
!pip install -q simpletransformers
!pip install numpy requests nlpaug

!pip install --q torch sentencepiece accelerate transformers pandas requests gradio bitsandbytes langchain faiss-gpu sentence-transformers gradio
!pip3 install tensorflow_text>=2.0.0rc0
!pip install keras-nlp
!pip install numba
!pip install tensorflow-addons

In [ ]:
#@title import library

import pandas as pd
import numpy as np
import re

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from transformers import pipeline

import nltk
import pythainlp
from pythainlp.util import normalize
from pythainlp.tokenize import subword_tokenize
from pythainlp.corpus import thai_stopwords

from flask import Flask, request
from pyngrok import ngrok
import copy

from linebot import  LineBotApi, WebhookHandler
from linebot.models import TextMessage, TextSendMessage

import openai

# OpenAi API
You can find your API key at https://platform.openai.com/account/api-keys.

In [ ]:
openai.api_key = '<openai_api_key_here>'
def open_domain(question_txt):
    messages = [{"role": "user", "content":question_txt}]
    chat = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
    ans_text = chat.choices[0].message.content
    return ans_text

In [ ]:
open_domain('สวัสดี')

# Models

In [ ]:
#@title product-shop information

df_product = pd.read_excel('dataset/product.xlsx')
shop_information = [
"ร้านเปิด-ปิดเวลา 9:00-20:00 ร้านหยุดทุกวันพุธ",
"เบอร์ติดต่อร้าน 099-999-9999",
"ร้านตั้งอยู่ที่ 'เลขที่ 148 ถนนเสรีไทย แขวงคลองจั่น เขตบางกะปิ กรุงเทพฯ 10240'",
"ร้าน 'THE CLOTHSET' ขายเสื้อผ้าสตรี",
]

def product_detail(product_name, shop_prob, shop_th=0.6):
    product_size_detail_top = """
ขนาดเสื้อ คือ
ไซส์ S   มีขนาดรอบอก 34 นิ้ว
ไซส์ M   มีขนาดรอบอก 36 นิ้ว
ไซส์ L   มีขนาดรอบอก 38 นิ้ว
ไซส์ XL  มีขนาดรอบอก 40 นิ้ว
ไซส์ XXL มีขนาดรอบอก 42 นิ้ว
ไซส์ XXL มีขนาดรอบอก 44 นิ้ว
"""
    product_size_detail_bottom = """
ขนาดกางเกงและกระโปรง คือ
ไซส์ S   มีขนาดรอบเอว 26 นิ้ว สะโพก 30 นิ้ว
ไซส์ M   มีขนาดรอบเอว 30 นิ้ว สะโพก 32 นิ้ว
ไซส์ L   มีขนาดรอบเอว 34 นิ้ว สะโพก 36 นิ้ว
ไซส์ XL  มีขนาดรอบเอว 33 นิ้ว สะโพก 40 นิ้ว
ไซส์ XXL มีขนาดรอบเอว 36 นิ้ว สะโพก 43 นิ้ว
"""
    answ = []
    count_product = df_product.loc[df_product['product']==product_name,'count'].item()
    if count_product == 0:
      answ.append('ขออภัยค่ะคุณลูกค้า ตอนนี้สินค้าหมดค่ะ')
    else:
      if shop_prob.loc[shop_prob['labels']=='size','scores'].item() >= shop_th:
        tmp = df_product.loc[df_product['product']==product_name,'size'].item()
        if 'เสื้อ' in product_name:
          answ.append('มีขนาด '+tmp+product_size_detail_top)
        else:
          answ.append('มีขนาด '+tmp+product_size_detail_bottom)
      if shop_prob.loc[shop_prob['labels']=='cost','scores'].item() >= shop_th:
        tmp = df_product.loc[df_product['product']==product_name,'price'].item()
        tmp1 = df_product.loc[df_product['product']==product_name,'promotion_type'].item()
        tmp2 = df_product.loc[df_product['product']==product_name,'promotion_price'].item()
        if tmp1 == '1 get 1':
          answ.append('ตอนนี้มีโปรโมชั่นซื้อ 1 แถม 1 ค่ะคุณลูกค้า หากมีข้อสงสัยเพิ่มเติมสามารถสอบถามได้เลยค่ะ')
        elif tmp1 == 'discount':
          answ.append(f'ตอนนี้มีโปรโมชั่นลดราคาจาก {tmp:d} เหลือเพียง {tmp2:d} บาท หากมีข้อสงสัยเพิ่มเติมสามารถสอบถามได้เลยค่ะ')
        answ.append('ราคา '+str(int(tmp))+' บาท')
      if shop_prob.loc[shop_prob['labels']=='material','scores'].item() >= shop_th:
        tmp = df_product.loc[df_product['product']==product_name,'material'].item()
        answ.append('ทำจาก'+tmp)
      if shop_prob.loc[shop_prob['labels']=='color','scores'].item() >= shop_th:
        tmp = df_product.loc[df_product['product']==product_name,'color'].item()
        answ.append('สินค้ามีสี '+tmp)
      if shop_prob.loc[shop_prob['labels']=='promotion','scores'].item() >= shop_th:
        tmp = """ตอนนี้มีโปรโมชั่นซื้อ 1 แถม 1 สำหรับเสื้อครอป
โปรโมชั่นลดราคาสำหรับกางเกงขาสั้นจาก 199 เหลือเพียง 149 บาท หากมีข้อสงสัยเพิ่มเติมสามารถสอบถามได้เลยค่ะ
        """
        answ.append(tmp)
    answ = '\n'.join(answ)
    if len(re.findall('หากมีข้อสงสัยเพิ่มเติมสามารถสอบถามได้เลยค่ะ',answ))>1:
      answ = answ.replace('หากมีข้อสงสัยเพิ่มเติมสามารถสอบถามได้เลยค่ะ','').strip()+' หากมีข้อสงสัยเพิ่มเติมสามารถสอบถามได้เลยค่ะ'
    elif (len(re.findall('หากมีข้อสงสัยเพิ่มเติมสามารถสอบถามได้เลยค่ะ',answ))==0) & ('ตอนนี้สินค้าหมดค่ะ' not in answ):
      answ = answ+'\nตอนนี้ยังมีสินค้าอยู่ลูกค้าสนใจรับเลยไหมคะ หรือหากมีข้อสงสัยเพิ่มเติมสามารถสอบถามได้เลยค่ะ'
    return answ

In [ ]:
#@title models

#intent classification
preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2", name = 'USE_PREPROCESSOR')
encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1", name = 'USE_ENDCODER')

label_dict = {0: 'ask_for_info', 1: 'comment', 2: 'greeting', 3: 'order', 4: 'shop_open'}
num_class = 5
i = tf.keras.layers.Input(shape=(), dtype=tf.string, name='TEXT_INPUT')
x = preprocessor(i)
x = encoder(x)
x = tf.keras.layers.Dropout(0.2, name="DROPOUT")(x['pooled_output'])
x = tf.keras.layers.Dense(num_class, activation='softmax', name="OUTPUT")(x)
model_classification = tf.keras.Model(i, x, name="USE_INTENT_CLASSIFICATION")
model_classification.load_weights("/content/Closed-Domain-Chatbot/weight/intent_classification_weight.hdf5")

#translator
translator_th_en = pipeline("translation", model="Helsinki-NLP/opus-mt-th-en")
#slots labeling
model_labeling = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
#sentiment analysis
comment_sentiment = pipeline('sentiment-analysis', 'airesearch/wangchanberta-base-att-spm-uncased', revision = 'finetuned@wisesight_sentiment')

In [ ]:
#@title chatbot
def get_answer(input_text):
    ###########################################################
    # text preparation
    ###########################################################
    clean_words_master = pd.read_excel('dataset/text_normalization.xlsx', sheet_name = 'clean_words')
    th_stopwords = pd.read_excel('dataset/stopwords.xlsx', sheet_name = 'stop_words')
    th_stopwords = th_stopwords['stop_words'].to_list()
    txt_normalize = normalize(input_text)
    for _,w in clean_words_master.iterrows():
    	txt_normalize = txt_normalize.replace(w['original'],w['clean'])
    txt_tokenize = subword_tokenize(txt_normalize, engine="wangchanberta")
    for i in txt_tokenize:
      if (i.replace('▁','') in th_stopwords) & (len(i.strip())>0):
        txt_rm_stopwords = txt_normalize.replace(i,'')
    for _,w in clean_words_master.iterrows():
    	txt_clean = txt_rm_stopwords.replace(w['original'],w['clean'])
    ###########################################################
    # intent classification
    ###########################################################
    question_txt = txt_clean
    answer_text = []
    overview_class = label_dict.get(np.argmax(model_classification.predict([question_txt])))
    ###########################################################
    # slots labeling for each class (sentiment analysis for comment class)
    ###########################################################
    ov_th = 0.4
    if (overview_class == "ask_for_info") | (model_classification.predict([question_txt])[0][0] > ov_th):
    	trans_to_en = translator_th_en(question_txt, src_lang='th', tgt_lang='en')
    	sequence_to_classify =  trans_to_en[0].get('translation_text')
    	shop_labels = ['size', 'cost', 'material', 'color','promotion', 'shop']
    	shop_prob_detail = model_labeling(sequence_to_classify, shop_labels, multi_label=True)
    	shop_prob_detail = pd.DataFrame(shop_prob_detail)[['labels','scores']]

    	th_1 = 0.6
    	if "เสื้อ" in question_txt:
    	  if ('เสื้อครอป' in question_txt) | ('ครึ่งตัว' in question_txt):
    		  answ = 'สินค้าเสื้อครอปครึ่งตัว '+product_detail('เสื้อครอปครึ่งตัว',shop_prob_detail, th_1)
    		  answer_text.append(answ)
    	  if 'เสื้อคลุม' in question_txt:
    		  answ = 'สินค้าเสื้อคลุม '+product_detail('เสื้อคลุม', shop_prob_detail, th_1)
    		  answer_text.append(answ)
    	  if (('เสื้อยืด' in question_txt) | ('เสื้อยืด' in question_txt)) & ('ครึ่งตัว' not in question_txt) & ('เสื้อครอป' not in question_txt) & ('เสื้อคลุม' not in question_txt):
    		  answ = 'สินค้าเสื้อยืดเต็มตัว '+product_detail('เสื้อยืดเต็มตัว', shop_prob_detail, th_1)
    		  answer_text.append(answ)
    	if "กางเกง" in question_txt:
    	  if 'กางเกงขายาว' in question_txt:
    		  answ = 'สินค้ากางเกงขายาว '+product_detail('กางเกงขายาว', shop_prob_detail, th_1)
    		  answer_text.append(answ)
    	  if 'กางเกงขาสั้น' in question_txt:
    		  answ = 'สินค้ากางเกงขาสั้น '+product_detail('กางเกงขาสั้น', shop_prob_detail, th_1)
    		  answer_text.append(answ)
    	if 'กระโปรง' in question_txt:
    	  if 'กระโปรงพลีท' in question_txt:
    		  answ = 'สินค้ากระโปรงพลีท '+product_detail('กระโปรงพลีท', shop_prob_detail, th_1)
    		  answer_text.append(answ)
    	  if 'กระโปรงทรงเอ' in question_txt:
    		  answ = 'สินค้ากระโปรงทรงเอ '+product_detail('กระโปรงทรงเอ', shop_prob_detail, th_1)
    		  answer_text.append(answ)

    	if shop_prob_detail.loc[shop_prob_detail['labels']=='shop','scores'].item() >= th_1:
    	  answ = """THE CLOTHSET ร้านขายเสื้อผ้าสำหรับสตรี
ทางร้านมีขายทั้งเสื้อครอปตัวสั้น, เสื้อยืด, เสื้อคลุม, กางเกงขายาว, กางเกงขาสั้น, กระโปรงพลีท, กระโปรงทรงเอ
หากสนใจสินค้าตัวไหนสามารถสอบถามได้เลยนะคะ"""
    	  answer_text.append(answ)
    	else:
    	  if len(answer_text)==0: #chatGPT
    		  answ = open_domain(question_txt)
    		  answer_text.append(answ)

    elif (overview_class == "shop_open") | (model_classification.predict([question_txt])[0][4] > ov_th):
    	trans_to_en = translator_th_en(question_txt, src_lang='th', tgt_lang='en')
    	sequence_to_classify =  trans_to_en[0].get('translation_text')
    	shop_labels = ['when', 'where', 'contact', 'shop']
    	shop_prob = model_labeling(sequence_to_classify, shop_labels, multi_label=True)
    	shop_prob = pd.DataFrame(shop_prob)[['labels','scores']]

    	shop_th = 0.6
    	if shop_prob.loc[shop_prob['labels']=='when','scores'].item() >= shop_th:
    	  answ = shop_information[0]
    	  answer_text.append(answ)
    	if shop_prob.loc[shop_prob['labels']=='contact','scores'].item() >= shop_th:
    	  answ = shop_information[1]
    	  answer_text.append(answ)
    	if shop_prob.loc[shop_prob['labels']=='where','scores'].item() >= shop_th:
    	  answ = shop_information[2]
    	  answer_text.append(answ)
    	if shop_prob.loc[shop_prob['labels']=='shop','scores'].item() >= shop_th:
    	  answ = """THE CLOTHSET ร้านขายเสื้อผ้าสำหรับสตรี
ทางร้านมีขายทั้งเสื้อครอปตัวสั้น, เสื้อยืด, เสื้อคลุม, กางเกงขายาว, กางเกงขาสั้น, กระโปรงพลีท, กระโปรงทรงเอ
หากสนใจสินค้าตัวไหนสามารถสอบถามได้เลยนะคะ"""
    	  answer_text.append(answ)
    	else:
    	  if len(answer_text)==0: #chatGPT
    		  answ = open_domain(question_txt)
    		  answer_text.append(answ)

    elif (overview_class == "comment") | (model_classification.predict([question_txt])[0][1] > ov_th):
      cm_class = comment_sentiment(question_txt)[0].get('label')
      if cm_class == 'neg':
    	  answ = 'ทางร้านขออภัยเป็นอย่างยิ่งในความผิดพลาดที่เกิดขึ้น ทางร้านจะพยายามทำการแก้ไขอย่างเร็วที่สุดค่ะ'
    	  answer_text.append(answ)
      elif cm_class == 'pos':
      	answ = 'ขอขอบคุณคุณลูกค้ามากนะคะ ที่ให้ความไว้วางใจ ทางร้านยินดีให้บริการเสมอค่ะ'
      	answer_text.append(answ)
      else:
      	if len(answer_text)==0: #chatGPT
      		answ = open_domain(question_txt)

    elif (overview_class == "greeting") | (model_classification.predict([question_txt])[0][2] > ov_th):
      trans_to_en = translator_th_en(question_txt, src_lang='th', tgt_lang='en')
      sequence_to_classify =  trans_to_en[0].get('translation_text')
      shop_labels = ['hello','thank']
      shop_prob_greeting = model_labeling(sequence_to_classify, shop_labels, multi_label=True)
      shop_prob_greeting = pd.DataFrame(shop_prob_greeting)[['labels','scores']]

      th_2 = 0.6
      if shop_prob_greeting.loc[shop_prob_greeting['labels']=='hello','scores'].item() >= th_2:
      	answ = 'สวัสดีค่ะ THE CLOTHSET ยินดีรับใช้บริการค่ะ หากสนใจสินค้าตัวไหนสามารถสอบถามได้เลยนะคะ'
      	answer_text.append(answ)
      elif shop_prob_greeting.loc[shop_prob_greeting['labels']=='thank','scores'].item() >= th_2:
      	answ = 'ขอขอบคุณคุณลูกค้ามากนะคะ ที่ให้ความไว้วางใจ ซื้อสินค้ากับทางร้าน THE CLOTHSET ยินดีรับใช้บริการค่ะ'
      	answer_text.append(answ)
      else:
      	answ = 'THE CLOTHSET ยินดีรับใช้บริการค่ะ'
      	answer_text.append(answ)
    elif (overview_class == "order") | (model_classification.predict([question_txt])[0][3] > ov_th):
      trans_to_en = translator_th_en(question_txt, src_lang='th', tgt_lang='en')
      sequence_to_classify =  trans_to_en[0].get('translation_text')
      shop_labels = ['interest','not interest','refund']
      shop_prob_order = model_labeling(sequence_to_classify, shop_labels, multi_label=True)
      shop_prob_order = pd.DataFrame(shop_prob_order)[['labels','scores']]

      th_3 = 0.6
      if shop_prob_order.loc[shop_prob_order['labels']=='interest','scores'].item() >= th_3:
      	answ = """ขอบพระคุณมาก ๆ ค่ะคุณลูกค้า ที่ให้ความไว้วางใจ ซื้อสินค้ากับทางร้าน THE CLOTHSET
ทางรบกวนขอชื่อที่อยู่และเบอร์ติดต่อ
โดยหากโอนเงินแล้วรบกวนแนบรูปสลิปด้วยนะคะ
โอนธนาคาร xxx เลขบัญชี 1234-567-890"""
      	answer_text.append(answ)
      elif shop_prob_order.loc[shop_prob_order['labels']=='not interest','scores'].item() >= th_3:
      	answ = 'รับทราบค่ะ หากมีข้อสงสัยเพิ่มเติม THE CLOTHSET ยินดีรับใช้บริการค่ะ'
      	answer_text.append(answ)
      elif shop_prob_order.loc[shop_prob_order['labels']=='refund','scores'].item() >= th_3:
      	answ = 'ขออภัยเป็นอย่างสูงค่ะคุณลูกค้า เดี๋ยวทางร้านจะรีบประสานงานให้นะคะ'
      	answer_text.append(answ)
      else:
      	answ = 'ขออภัยค่ะ เดี๋ยวทางร้านจะรีบประสานงานให้นะคะ'
      	answer_text.append(answ)
    ######################################
    answer_clean = '\n'.join(answer_text)
    return answer_clean

# Chatbot result

**example result**

In [ ]:
print(get_answer('ร้านเปิดตอนหนัยบ้างอ่าคะ พอดีอยากทราบเบอร์ติดต่ออ่ะค่ะๆ'))

**connect line**

In [ ]:
channel_access_token = '<channel_access_token_here>'
channel_secret = '<channel_secret_here>'
ngrok_authen_token = '<ngrok token>'

In [ ]:
port_no = 5000

#Flask
app = Flask(__name__)
ngrok.set_auth_token(ngrok_authen_token)
public_url =  ngrok.connect(port_no).public_url

line_bot_api = LineBotApi(channel_access_token)
handler = WebhookHandler(channel_secret)

@app.route("/callback", methods=['POST'])
def callback():

    req = request.get_json(silent=True, force=True)

    input_msg = req['events'][0]['message']['text']
    reply_token = req['events'][0]['replyToken']
    user_id = req['events'][0]['source']['userId']

    ans_text = get_answer(input_msg)

    output_msg = TextSendMessage(text=ans_text)
    line_bot_api.push_message(user_id, output_msg)

#Flask
if __name__ == '__main__':
    public_url_s = public_url.replace("http://","https://")
    print(f"To acces the Gloable link please click {public_url_s}/callback")
    print("Starting app on port %d" % port_no)
    app.run(debug=False, port=port_no, threaded=True)